# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [16]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [17]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Sit', 'Left', 'Knee', 'Frog', 'Right', 'Crouch', 'StandInit', 'Stand', 'Back', 'Belly', 'HeadBack']


In [18]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []

    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []


for i in range(len(classes)):
    temp_data, temp_target = load_pose_data(i)
    for val in enumerate(temp_data):
        all_data.append(val[1])
    for val in enumerate(temp_target):
        all_target.append(val[1])         
        
all_data_array = np.ndarray((len(all_data[0]),len(all_data)))    
all_target_array = np.ndarray(len(all_target))   

for i in range(len(all_data)):
    all_data_array[:,i] = all_data[i] 
    
for i in range(len(all_target)):
    all_target_array[i] = all_target[i] 

all_data_array = all_data_array.T

all_data = all_data_array
all_target = all_target_array

In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
predicting_data = list(set(permutation) - set(training_data))

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:
clf.fit(all_data[training_data], all_target[training_data])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [8]:
predicted_result, target__result = clf.predict(all_data[predicting_data]), all_target[predicting_data]

In [9]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [10]:
expected = []
predicted = []

expected = target__result
predicted = predicted_result

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00        14
        1.0       1.00      1.00      1.00         5
        3.0       1.00      1.00      1.00         1
        4.0       1.00      1.00      1.00         4
        5.0       1.00      1.00      1.00         7
        6.0       1.00      1.00      1.00        18
        7.0       1.00      1.00      1.00         6
        8.0       1.00      1.00      1.00         4
        9.0       1.00      1.00      1.00         7
       10.0       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        67


Confusion matrix:
[[14  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0  6  0  0  0]
 [ 0  0  0  0  0  0  0  4  0  0]
 [ 0  0  0  0  0  0  0  0  7  0]
 [ 0  0  0  0  0  0  0

## 4. Evaluate on the test data

In [12]:
expected = []
predicted = []

expected = target__result
predicted = predicted_result

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00        14
        1.0       1.00      1.00      1.00         5
        3.0       1.00      1.00      1.00         1
        4.0       1.00      1.00      1.00         4
        5.0       1.00      1.00      1.00         7
        6.0       1.00      1.00      1.00        18
        7.0       1.00      1.00      1.00         6
        8.0       1.00      1.00      1.00         4
        9.0       1.00      1.00      1.00         7
       10.0       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        67


Confusion matrix:
[[14  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0  6  0  0  0]
 [ 0  0  0  0  0  0  0  4  0  0]
 [ 0  0  0  0  0  0  0  0  7  0]
 [ 0  0  0  0  0  0  0

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [13]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [14]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[predicting_data]), all_target[predicting_data]

(array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          9.,   0.,   0.,   6.,   0.,   1.,   8.,   6.,   1.,   6.,   7.,
          1.,   7.,   7.,   1.,   1.,   6.,   8.,   6.,   8.,   6.,   3.,
          8.,  10.,   9.,   7.,   4.,   4.,   6.,   4.,   9.,   4.,   5.,
          6.,   5.,   9.,   7.,   5.,   5.,   7.,   9.,   5.,   9.,   5.,
          5.,   6.,   6.,   6.,   6.,   9.,   6.,   6.,   6.,   6.,   6.,
          6.]),
 array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          9.,   0.,   0.,   6.,   0.,   1.,   8.,   6.,   1.,   6.,   7.,
          1.,   7.,   7.,   1.,   1.,   6.,   8.,   6.,   8.,   6.,   3.,
          8.,  10.,   9.,   7.,   4.,   4.,   6.,   4.,   9.,   4.,   5.,
          6.,   5.,   9.,   7.,   5.,   5.,   7.,   9.,   5.,   9.,   5.,
          5.,   6.,   6.,   6.,   6.,   9.,   6.,   6.,   6.,   6.,   6.,
          6.]))